# Project 2: Republication report 

## Introduction

In this project, we attempt replicating the figure 2.a from the article "Friendship and Mobility: User Movement in Location-Based Social Networks ". The figure represents the relation between the probability of friendship and the distance between their houses. To replicate it, we will first find the position of the houses of each user by using the method described in the article and then assign a distance between each user and his friends. Finally, we will be able to calculate the probability of occurrence for all the distance and replicate the figure. More details are provided in the following sections. The article can be found at this address: https://drive.google.com/drive/folders/1QRoC6DAMoD_BxJ6KPMdijBRTDUYhoVfG?usp=sharing

## Importing packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # needed for plotting
import geopandas as gpd # for Geo-location filtering
from netCDF4 import Dataset as nc  # for loading standard climate date format (nc extension)
import netCDF4 # for loading standard climate date format (nc extension)
import xarray as xr # for handling climate data
from pandarallel import pandarallel # for running pandas functions in parallel
import multiprocessing # for general parallelizing of codes
import tqdm # for having progres bar
pandarallel.initialize(nb_workers=multiprocessing.cpu_count()-1)
from geopandas.tools import sjoin
from tqdm import tqdm # package used for having progres bar
tqdm.pandas()
from datetime import datetime, date # package to thandle date objects
from scipy import stats # used for statistic analysis
import holidays # package used to obtain holidays
from timezonefinder import TimezoneFinderL # package used to convert time accroding to timezones
tf = TimezoneFinderL(in_memory=False)
from pytz import timezone # package used to convert to recognize timezones from location
import pytz
utc = pytz.utc

## Some notes
### Pandarallel and multiprocessing
1. considering the large datasets that we deal with (e.g. temperature by date in each location on US which is a 3d grided data) we were urged to run our analysis and especially data wrangling using full power of our machine up to its capacity. Therefore, we are runing our operations in parallel and ```multiprocessing``` and ```pandarallel``` were used to do so 
2. The ```NetCDF``` package is inprted to handle ***nc*** file extension which is the standard extension for historical climate data. Our data was obtained from:

     https://www.northwestknowledge.net/metdata/ 

     which is provided by University of Idaho. The data needs to be processed before being used in our analysis.

3. ```xarray``` package is used to handle multi-dimensional arrays such as climate data

4. ```tqdm``` let's us have cool progress bat on our loops

5. ```timezonefinder``` and ```pytz``` are used to handle time zones

## Loading the data

In [ ]:
# Data of two set of data
DATA_FOLDER = 'Data/'

INP_FOLDER = 'InputData/'

# Friendship undirected network users and Time and location information of check-ins made by users

#Brightkite
BRIGHT_NETWORK_DATASET = DATA_FOLDER + "Brightkite_edges.txt.gz"
BRIGHT_CHECKIN_TIME_LOCATION_DATASET = DATA_FOLDER + "Brightkite_totalCheckins.txt.gz"

#Gowalla
GOWALLA_NETWORK_DATASET = DATA_FOLDER + "Gowalla_edges.txt.gz"
GOWALLA_CHECKIN_TIME_LOCATION_DATASET = DATA_FOLDER + "Gowalla_totalCheckins.txt.gz"

# loading all the data set
b_network_df = pd.read_csv(BRIGHT_NETWORK_DATASET, delimiter="\t",
                                    error_bad_lines =False, header = 0 )
b_checkin_df = pd.read_csv(BRIGHT_CHECKIN_TIME_LOCATION_DATASET, delimiter="\t",
                                       error_bad_lines =False, header = 0 )
g_network_df = pd.read_csv(GOWALLA_NETWORK_DATASET, delimiter = "\t",header = None)
g_checkin_df = pd.read_csv(GOWALLA_CHECKIN_TIME_LOCATION_DATASET, delimiter = "\t",header = None)

In [ ]:
#Set up the header 
NETWORK_COLUMNS = ['user','friend_edge']
CHECKIN_COLUMNS = ['user','checkin_time','latitude','longitude','location_id']

In [ ]:
#Rename the columns
b_network_df.columns = NETWORK_COLUMNS
b_checkin_df.columns = CHECKIN_COLUMNS
g_network_df.columns = NETWORK_COLUMNS
g_checkin_df.columns = CHECKIN_COLUMNS

In [ ]:
b_network_df.sample(3)

In [ ]:
b_checkin_df.sample(3)

In [ ]:
g_network_df.sample(3)

In [ ]:
g_checkin_df.sample(3)

---------------------------

## Pre-process

At first, before starting our recollection, we need to pre-process our dataset. So, we can begin by counting all the null values and remove them if necessary. We also want the latitude and longitude to be respectively in the interval [-90,90] and [-180,180].

In [ ]:
g_checkin_df.isnull().sum()

In [ ]:
b_checkin_df.isnull().sum()

In [ ]:
'''Cleaning the data'''

# locate rows with NaN
rows_with_NaN_b = b_checkin_df.loc[(b_checkin_df['latitude'].isna() == True)]
rows_with_NaN_g = g_checkin_df.loc[(g_checkin_df['latitude'].isna() == True)]

# locate rows that are not in those interval : -90 < Latitude < 90 and -180 < Longitude < 180

# latitude
lat_to_removed_b = b_checkin_df.loc[(b_checkin_df['latitude'] > 90) | (b_checkin_df['latitude'] < -90)]
lat_to_removed_g = g_checkin_df.loc[(g_checkin_df['latitude'] > 90) | (g_checkin_df['latitude'] < -90)]

# longitude
long_to_removed_b = b_checkin_df.loc[(b_checkin_df['longitude'] > 180) | (b_checkin_df['longitude'] < -180)]
long_to_removed_g = g_checkin_df.loc[(g_checkin_df['longitude'] > 180) | (g_checkin_df['longitude'] < -180)]

# Dropping all the rows

# Brightkite
b_checkin_df = b_checkin_df.drop(rows_with_NaN_b.index)
b_checkin_df = b_checkin_df.drop(lat_to_removed_b.index)
b_checkin_df = b_checkin_df.drop(long_to_removed_b.index)

#see bellow
b_checkin_df = b_checkin_df.drop(b_checkin_df[b_checkin_df['location_id'] == '00000000000000000000000000000000'].index)

# Gowalla
g_checkin_df = g_checkin_df.drop(rows_with_NaN_g.index)
g_checkin_df = g_checkin_df.drop(lat_to_removed_g.index)
g_checkin_df = g_checkin_df.drop(long_to_removed_g.index)

Note that some weird values were spotted for the bright kite dataset on the position (0,0) with a "location_id" equal to " 00000...0000 ". As quoted on page 1083 of our article: " The total number of check-ins for Gowalla is 6.4 million and 4.5 million for Brightkite. " So to be more consistent with those numbers we decide to also remove them. We can now show the shape of our two datasets.

In [ ]:
# Number of the check-ins used in the recollection 
print('The total number of check-ins for Gowalla :',g_checkin_df.shape[0])
print('The total number of check-ins for Brightkite :',b_checkin_df.shape[0])

-------------------------------------------------------------------------

## Data Recollection 

### Classing every check-ins in cells of 25x25 km 
In the next two sections, we will provide more details on the method described in the paper.
So, we begin by classing every check-in in a grid of 25x25 km over the world. One way of classing into cells is to perform euclidean division by 25 on both latitude and longitude. Then we can use the quotient of those divisions to identify our cells.
As we have two dimensions, we find two sets of intervals ( one on latitude and one on longitude) with the origin of the gird on (0,0). Both sets can be either positive or negative to map all the four earth's dial. Thus, two "check-ins" with the same latitude and longitude intervals will be classified in the same cell. Note that to do a euclidean division by 25 km, we need to transform all latitudes and longitudes into kilometers and take into account the distortion of the earth. This is performed by two functions which follow the formula from this website :
https://stackoverflow.com/questions/1253499/simple-calculations-for-working-with-lat-lon-and-km-distance

In [ ]:
def lat_degree_to_km(latitude):
    '''
    Input : Latitude in degree
    Output : Latitude in km
    
    '''
    return latitude * 110.574
    

In [ ]:
def long_degree_to_km(latitude,longitude):
    '''
    Input : Longitude , Latitude in degree
    Output : Longitude in km  
    
    '''
    return 111.320 * np.cos((latitude * np.pi) / 180) * longitude

In [ ]:
def classing_into_cells(g_checkin_df):
    '''
    Input : the time and location information of check-ins made by users
    Output : check-in dataset classified by cells
    
    '''
    # Creation of columns regrouping km for longitude and latitude with the two function see above
    
    g_checkin_df['lat_km'] = lat_degree_to_km(g_checkin_df['latitude']) 
    g_checkin_df['long_km'] = long_degree_to_km(g_checkin_df['latitude'],g_checkin_df['longitude'])
    
    # creation of cells with size 25 x 25 km represented by two sets of intervals int_longitude and int_latitude. 
    # By taking those two together we can build a grid with an origin at (0,0)
    # Note that when the interval is transfering from float to int he takes the floor number for 
    # positive number and the ceil number for the negative interval, that's what we want .
    
    g_checkin_df['int_lat'] = (g_checkin_df['lat_km'] / 25).astype(int)
    g_checkin_df['int_long'] = (g_checkin_df['long_km'] / 25).astype(int)
    
    return g_checkin_df

In [ ]:
g_checkin_df = classing_into_cells(g_checkin_df)
b_checkin_df = classing_into_cells(b_checkin_df)

In [ ]:
g_checkin_df.sample(3)

In [ ]:
b_checkin_df.sample(3)

-----------------------------------------

### Finding the location of the users (latitude, longitude)

Now we want to find the location of the user's houses. Thus, we assume that the position of the house of the user will be at the mean of all the check-ins positions (latitude and longitude) appearing in one particular cell.
This particular cell is established by analyzing all cells linked to one user and pick the one that has the most number of check-ins in it. The article claims that this method is known to be accurate at 85 %.

##### Method:
Here, we use the function "groupby" to get a new dataset that grouped for each user all his check-ins by cells by counting how many rows appear to have the same "int_lat", "int_long" and "user". So then, we can sort all counts by users to take only the top one with the highest number of check-ins. We merge this dataset to the one classified by cells. Then, by comparing all check-ins cells and the previous result we can find the rows representing the cell with the most check-ins. Finally, the location of the house will be the average position of all the check-ins contained in this particular cell. 

Note that in the paper "Friendship and Mobility: User Movement in Location-Based Social Networks", any information was provided on how to deal with users that had the same number of check-ins on all his cells. For them, we decide to just take the cell located at the top of the user's groupby after sorting it without any further research.


In [ ]:
def find_user_house(g_checkin_df):
    '''
    Input : check-in dataset classified by cells
    Output : dataset link each user's with his house location (latitude, longitude)
    
    '''
    # finding intervals that has the highest number of check-ins in it
    
    int_house_users = g_checkin_df.groupby(['user','int_lat','int_long']).count()
    int_house_users = int_house_users['checkin_time'].sort_values(
        ascending = False).groupby(level = 0).head(1).reset_index()
    int_house_users = int_house_users.rename(
        {'int_lat' : 'house_int_latitude','int_long' : 'house_int_longitude'},
        axis = "columns")
    
    # finding all the check-ins made in those intervals (cells)
    
    house_users = g_checkin_df.merge(int_house_users,
                    left_on = ['user'],
                    right_on = ['user'])
    
    house_users = house_users.loc[(house_users['int_lat'] == house_users['house_int_latitude'])
                            & (house_users['int_long'] == house_users['house_int_longitude'])]
    
    #finding latitude and longitude of the user's house.
    
    houses = house_users.groupby(['user']).mean()[['latitude','longitude']].rename(
        { 'latitude':'user_house_lat' ,'longitude':'user_house_long'},
        axis = "columns")
    return houses

In [ ]:
houses_gowalla = find_user_house(g_checkin_df)
houses_bright = find_user_house(b_checkin_df)

----------------------

### Matching the houses of the users and their friends 

By using the result from the last section, we have at our disposal two data set which maps all users with the location of their houses. In addition to this, we also have the two Friendship undirected network initially provided. As stated in the introduction, our goal is to find the distance between each user and his friends. In order to do that we match the two networks ("g_network_df" and "b_network_df") with both the location of their user's houses and the location of the friend's houses in one dataset. Then, we can calculate the distance for each row by the following method of the sinus:
http://villemin.gerard.free.fr/aGeograp/Distance.htm

In [ ]:
def dist_km(Lat_start,Long_start,Lat_end,Long_end):
    '''
    Input : latitude and longitude of a point A and B
    Output : Distance between A and B
    
    '''
    # Calcul with the method of sinus
    distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
                          + np.cos(np.pi * Lat_start / 180.0) * np.cos(np.pi * Lat_end / 180.0) \
                            * np.cos(np.pi * Long_start / 180.0 - np.pi * Long_end / 180.0)) * 6371
    return distance_km

In [ ]:
def matching_user_friend(houses,network_df):
    '''  
    
    Input: two dataset one containing the dataset which link each user's with his house location (latitude, longitude)
    and one other containing the friendship network of users
    Output: one dataset with the location of both users and their friends and with the distance linking the both houses
    
    '''
    
    # merge the location of the friends houses to the network data set
    
    friend_house_location = houses.reset_index().rename({'user':'friend_edge'},axis = 'columns')
    friend_house_location = network_df.merge(friend_house_location,
                                           on = 'friend_edge').rename(
        {'user_house_lat' : 'friend_house_lat','user_house_long' : 'friend_house_long'},
        axis = "columns")
    
    # merge the location of the users houses to this same dataset

    network_houses_locations_df = friend_house_location.merge(houses, on ='user')
    
    # calculate the distance between each users and their friends
    
    network_houses_locations_df['dist_btw_user_friend'] = dist_km(network_houses_locations_df['user_house_lat'],
                                                                  network_houses_locations_df['user_house_long']
                                                              ,network_houses_locations_df['friend_house_lat'],
                                                                  network_houses_locations_df['friend_house_long'])
    
    return network_houses_locations_df

In [ ]:
g_network_houses_locations_df = matching_user_friend(houses_gowalla, g_network_df)

In [ ]:
b_network_houses_locations_df = matching_user_friend(houses_bright,b_network_df)
# Here we have an error for the rows that are combining a user and a friend that live in the same cell so
# we are simply replacing the Nan values by zero to take those values into account.
b_network_houses_locations_df['dist_btw_user_friend'].loc[b_network_houses_locations_df['dist_btw_user_friend'].isna()] = 0

In [ ]:
g_network_houses_locations_df.sample(3)

In [ ]:
b_network_houses_locations_df.sample(3)

------------------------------

-----------------------------------------------------

# Adding extra data

In [ ]:
houses_gowalla = houses_gowalla.reset_index()
houses_bright = houses_bright.reset_index()

Making geodataframe with the user home

In [ ]:
gdf_home = gpd.GeoDataFrame(
    houses_gowalla, geometry=gpd.points_from_xy(houses_gowalla.user_house_long,houses_gowalla.user_house_lat))
gdf_home_b = gpd.GeoDataFrame(
    houses_bright, geometry=gpd.points_from_xy(houses_bright.user_house_long,houses_bright.user_house_lat))

In [ ]:
gdf_home.sample(3)

Import the shape file from https://www.igismap.com/united-states-shapefile-download-free-map-boundary-states-and-county/ called Download Polygon Shapefile of United States of America you can find the link for the download here https://map.igismap.com/share-map/export-layer/Alabama_AL4_US_Poly/1534b76d325a8f591b52d302e7181331

In [ ]:
#import shp
states =gpd.read_file(INP_FOLDER + 'United_States-_States_Polygon.shp') 

Then we can join our user by states

In [ ]:
# setting coordinate for sake of geopandas
gdf_home.crs = "EPSG:4326"
gdf_home_b.crs = "EPSG:4326"

In [ ]:
# here we append GeoDataFrame of users house to checkin dataframe
def sjoin_chunk(chunk):
    chunk.crs = "EPSG:4326"
    return sjoin(chunk, states, how='left')

def parallelize_dataframe_func(df, func):
    num_cores = multiprocessing.cpu_count()-1  #leave one free to not freeze machine
    num_partitions = num_cores #number of partitions to split dataframe
    df_split = np.array_split(df, num_partitions)
    pool = multiprocessing.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

pointInStates = parallelize_dataframe_func(gdf_home, sjoin_chunk)
pointInStates_b = parallelize_dataframe_func(gdf_home_b, sjoin_chunk)

We clear the user outside the USA

In [ ]:
# keeping data for users that reside in USA
pointInStates = pointInStates.loc[pointInStates['country']=='USA']
pointInStates_b = pointInStates_b.loc[pointInStates_b['country']=='USA']

Data cleaning to have only what we want

In [ ]:
pointInStates = pointInStates[['user','user_house_lat','user_house_long','name']]
pointInStates_b = pointInStates_b[['user','user_house_lat','user_house_long','name']]
user_in_usa_g = pointInStates['user']
user_in_usa_b = pointInStates_b['user']
gdf_home_plot = gpd.GeoDataFrame(pointInStates, geometry=gpd.points_from_xy(pointInStates.user_house_long,pointInStates.user_house_lat))
gdf_home_plot_b = gpd.GeoDataFrame(pointInStates_b, geometry=gpd.points_from_xy(pointInStates_b.user_house_long,pointInStates_b.user_house_lat))

In [ ]:
pointInStates.sample(3)

In [ ]:
# Taking only the major states
# list of states (for visualisation)
list_major_states =["Alabama","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]
point_in_major_states = gdf_home_plot.loc[gdf_home_plot['name'].isin(list_major_states)==True]
point_in_major_states_b= gdf_home_plot_b.loc[gdf_home_plot_b['name'].isin(list_major_states)==True]

major_states =states.loc[states['name'].isin(list_major_states)==True]


In [ ]:
# Here we plot the location of users' house over USA map; however we have prepared better results for the data story

ax = major_states['geometry'].plot(cmap='Greys',figsize=(25, 20),edgecolor='k')
point_in_major_states['geometry'].plot(ax=ax,color='r',markersize =4)
point_in_major_states_b['geometry'].plot(ax=ax,color='b',markersize =4)
plt.title('\n Major states of U.S with the user home location \n ',size=40)
plt.xlabel('\n Longitude \n ',size=40)
plt.ylabel('\n Latitude \n',size=40)
plt.xticks(size=20)
plt.yticks(size=20)
plt.legend(['Home Gowalla user','Home Brightkite user'],fontsize=20,loc='lower right')

In [ ]:
gdf_checkin = gpd.GeoDataFrame(
    g_checkin_df, geometry=gpd.points_from_xy(g_checkin_df.longitude,g_checkin_df.latitude))
gdf_checkin_b = gpd.GeoDataFrame(
    b_checkin_df, geometry=gpd.points_from_xy(b_checkin_df.longitude,b_checkin_df.latitude))

### Checkin only in the usa made by usa user

In [ ]:
gdf_checkin_usa_g = point_in_major_states.merge(gdf_checkin,left_on = ['user'],right_on = ['user'])
gdf_checkin_usa_b = point_in_major_states_b.merge(gdf_checkin_b,left_on = ['user'],right_on = ['user'])

### Cleaning

In [ ]:
# cleaning and reordering data
COLUMN_TO_DROP = ['lat_km','long_km','int_lat','int_long','geometry_y']
gdf_checkin_usa_g = gdf_checkin_usa_g.drop(COLUMN_TO_DROP,axis=1)
gdf_checkin_usa_b = gdf_checkin_usa_b.drop(COLUMN_TO_DROP,axis=1)
gdf_checkin_usa_g = gdf_checkin_usa_g.rename({'name':'States of homes'
                                             ,'geometry_x':'geometry homes'},axis=1)
gdf_checkin_usa_b = gdf_checkin_usa_b.rename({'name':'States of homes'
                                             ,'geometry_x':'geometry homes'},axis=1)

In [ ]:
# making the geoframedata of checkin locations
gdf_checkin_usa_g = gpd.GeoDataFrame(
    gdf_checkin_usa_g, geometry=gpd.points_from_xy(gdf_checkin_usa_g.longitude,
                                                   gdf_checkin_usa_g.latitude))
gdf_checkin_usa_b = gpd.GeoDataFrame(
    gdf_checkin_usa_b, geometry=gpd.points_from_xy(gdf_checkin_usa_b.longitude,
                                                   gdf_checkin_usa_b.latitude))

In [ ]:
# appending the geoframedata of checkin locations
check_in_pointInStates_b = parallelize_dataframe_func(gdf_checkin_usa_b, sjoin_chunk)
# reordering data
check_in_pointInStates_b = check_in_pointInStates_b[['user','user_house_lat',
                                                     'user_house_long','States of homes',
                                                     'geometry homes',
                                                     'checkin_time','latitude','longitude',
                                                     'country','name']]

In [ ]:

check_in_pointInStates_g = parallelize_dataframe_func(gdf_checkin_usa_g, sjoin_chunk)
# reordering data
check_in_pointInStates_g = check_in_pointInStates_g[['user','user_house_lat','user_house_long','States of homes','geometry homes',
                                                     'checkin_time','latitude','longitude','country','name']]

In [ ]:
# adding time as datetime object to the dataframe
check_in_pointInStates_b['date'] =\
check_in_pointInStates_b['checkin_time'].astype(str).parallel_apply(lambda x: np.datetime64(x.split("T", 1)[0]))

check_in_pointInStates_g['date'] =\
check_in_pointInStates_g['checkin_time'].astype(str).parallel_apply(lambda x: np.datetime64(x.split("T", 1)[0]))

In [ ]:
print(f"{len(check_in_pointInStates_g)=}")
print(f"{len(check_in_pointInStates_b)=}")

In [ ]:
check_in_pointInStates_g.sample()

In [ ]:
# Here we load the prepared ghcn (Global Historical Climatology Network) data which are:
# 1. minimum temperature
# 2. maximum temperature
# 3. precipitation
# over a day
ds_tmin = xr.open_dataset("./InputData/coarse_tmmn.nc")
ds_tmax = xr.open_dataset("./InputData/coarse_tmmx.nc")
ds_pr = xr.open_dataset("./InputData/coarse_pr.nc")

In the following handful of cells we assign climatic data for each checkin for both users' house location and the checkin location with looking up from the grided data that we have in the ***xarray*** form the ***nc*** files loaded above. It works such that for every checkin we look up the climate data that we have for that date and we try to find the nearest grid point that we have to that location in our dataset and we take the value of the grid point that we have and assign it to our checkin.

***Note***: we do not have climate data for the checkins outside of US an we will have some NaNs  in the climate data columns in our dataframe

In [ ]:
check_in_pointInStates_b["tmin_house"] =\
check_in_pointInStates_b.parallel_apply(
    lambda df : ds_tmin.sel(day=df.date, lon=df.user_house_long,
                            lat=df.user_house_lat, method="nearest",
                            tolerance=400).get('air_temperature').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_b["tmax_house"] =\
check_in_pointInStates_b.parallel_apply(
    lambda df : ds_tmax.sel(day=df.date, lon=df.user_house_long,
                            lat=df.user_house_lat, method="nearest",
                            tolerance=400).get('air_temperature').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_b["pr_house"] =\
check_in_pointInStates_b.parallel_apply(
    lambda df : ds_pr.sel(day=df.date, lon=df.user_house_long,
                          lat=df.user_house_lat, method="nearest",
                          tolerance=400).get('precipitation_amount').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_b["tmin_checkin"] =\
check_in_pointInStates_b.parallel_apply(
    lambda df : ds_tmin.sel(day=df.date, lon=df.longitude,
                            lat=df.latitude, method="nearest",
                            tolerance=400).get('air_temperature').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_b["tmax_checkin"] =\
check_in_pointInStates_b.parallel_apply(
    lambda df : ds_tmax.sel(day=df.date, lon=df.longitude,
                            lat=df.latitude, method="nearest",
                            tolerance=400).get('air_temperature').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_b["pr_checkin"] =\
check_in_pointInStates_b.parallel_apply(
    lambda df : ds_pr.sel(day=df.date, lon=df.longitude,
                          lat=df.latitude, method="nearest",
                          tolerance=400).get('precipitation_amount').data.ravel()[0],
    axis=1)

In [ ]:
print(f"{len(check_in_pointInStates_b)=}")

In [ ]:
check_in_pointInStates_b.sample(3)

In [ ]:
check_in_pointInStates_g["tmin_house"] =\
check_in_pointInStates_g.parallel_apply(
    lambda df : ds_tmin.sel(day=df.date, lon=df.user_house_long,
                            lat=df.user_house_lat, method="nearest",
                            tolerance=400).get('air_temperature').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_g["tmax_house"] =\
check_in_pointInStates_g.parallel_apply(
    lambda df : ds_tmax.sel(day=df.date, lon=df.user_house_long,
                            lat=df.user_house_lat, method="nearest",
                            tolerance=400).get('air_temperature').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_g["pr_house"] =\
check_in_pointInStates_g.parallel_apply(
    lambda df : ds_pr.sel(day=df.date, lon=df.user_house_long,
                            lat=df.user_house_lat, method="nearest",
                            tolerance=400).get('precipitation_amount').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_g["tmin_checkin"] =\
check_in_pointInStates_g.parallel_apply(
    lambda df : ds_tmin.sel(day=df.date, lon=df.longitude,
                            lat=df.latitude, method="nearest",
                            tolerance=400).get('air_temperature').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_g["tmax_cehckin"] =\
check_in_pointInStates_g.parallel_apply(
    lambda df : ds_tmax.sel(day=df.date, lon=df.longitude,
                            lat=df.latitude, method="nearest",
                            tolerance=400).get('air_temperature').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_g["pr_checkin"] =\
check_in_pointInStates_g.parallel_apply(
    lambda df : ds_pr.sel(day=df.date, lon=df.longitude,
                            lat=df.latitude, method="nearest",
                            tolerance=400).get('precipitation_amount').data.ravel()[0],
    axis=1)

In [ ]:
check_in_pointInStates_g.sample(5)

## Holidays

In [ ]:
us_holidays = holidays.UnitedStates()

In [ ]:
check_in_pointInStates_g['date'] = pd.to_datetime(check_in_pointInStates_g['date'], format = '%Y-%m-%d')
check_in_pointInStates_b['date'] = pd.to_datetime(check_in_pointInStates_b['date'], format = '%Y-%m-%d')

In [ ]:
check_in_pointInStates_g['us_holiday'] = check_in_pointInStates_g['date'].parallel_apply(us_holidays.get)

In [9]:
check_in_pointInStates_b['us_holiday'] = check_in_pointInStates_b['date'].parallel_apply(us_holidays.get)

In [9]:
# let's save the dataframes with added data into disk
check_in_pointInStates_g.to_csv('home_and_checkin_with_states_gowalla.csv')
check_in_pointInStates_b.to_csv('home_and_checkin_with_states_bright.csv')

---

# Metrics 
The metric that we use as evaluations of the checkins

#### Copy() or Load data to df_bright and df_gowalla

In [3]:
# Here we get ready calculating our metrics:
#df_df_brightt = check_in_pointInStates_b.copy()
#df_gowalla = check_in_pointInStates_g.copy()
df_bright = pd.read_csv('home_and_checkin_with_states_bright.csv')
df_gowalla = pd.read_csv('home_and_checkin_with_states_gowalla.csv')

In [4]:
len(df_bright['user'].unique())

29881

In [5]:
len(df_gowalla['user'].unique())

49605

## Seasons

Northern Hemisphere for local time of Denver, CO.
https://www.calendardate.com/year2009.php

In [10]:
def seasons(date):
    
    spring_2008 = datetime.strptime('2008-03-19', '%Y-%m-%d')
    summer_2008 = datetime.strptime('2008-06-20', '%Y-%m-%d')
    autumn_2008 = datetime.strptime('2008-09-22', '%Y-%m-%d')
    winter_2008 = datetime.strptime('2008-12-21', '%Y-%m-%d')
    spring_2009 = datetime.strptime('2009-03-20', '%Y-%m-%d')
    summer_2009 = datetime.strptime('2009-06-20', '%Y-%m-%d')
    autumn_2009 = datetime.strptime('2009-09-22', '%Y-%m-%d')
    winter_2009 = datetime.strptime('2009-12-21', '%Y-%m-%d')
    spring_2010 = datetime.strptime('2010-03-20', '%Y-%m-%d')
    summer_2010 = datetime.strptime('2010-06-21', '%Y-%m-%d')
    autumn_2010 = datetime.strptime('2010-09-22', '%Y-%m-%d')
    winter_2010 = datetime.strptime('2010-12-21', '%Y-%m-%d')
    
    if spring_2008 >= date:
        return 'winter_2007'
    elif summer_2008 >= date:
        return 'spring_2008'
    elif autumn_2008 >= date:
        return 'summer_2008'
    elif winter_2008 >= date:
        return 'autumn_2008'
    elif spring_2009 >= date:
        return 'winter_2008'
    elif summer_2009 >= date:
        return 'spring_2009'
    elif autumn_2009 >= date:
        return 'summer_2009'
    elif winter_2009 >= date:
        return 'autumn_2009'
    elif spring_2010 >= date:
        return 'winter_2010'
    elif summer_2010 >= date:
        return 'spring_2010'
    elif autumn_2010 >= date:
        return 'summer_2010' 
    elif winter_2010 >= date:
        return 'autumn_2010'
    else:
        return np.nan

In [11]:
df_gowalla['season'] = df_gowalla['date'].parallel_apply(seasons)
df_bright['season'] = df_bright['date'].parallel_apply(seasons)

## Distance From Home

In [16]:
df_gowalla['distance_from_home'] = df_gowalla.parallel_apply(lambda x: dist_km(x['user_house_lat'],\
                                                                     x['user_house_long'],\
                                                                     x['latitude'],\
                                                                     x['longitude']), axis=1)

<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arcco

In [17]:
df_bright['distance_from_home'] = df_bright.parallel_apply(lambda x: dist_km(x['user_house_lat'],\
                                                                     x['user_house_long'],\
                                                                     x['latitude'],\
                                                                     x['longitude']), axis=1)

<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arcco

## Distance between user check-ins

In [18]:
def offset(data):
    data_reset = data.reset_index(drop=True)
    data_offset = data_reset.set_index(np.arange(1, data_reset.shape[0]+1, 1))[['latitude','longitude']]
    return data_reset.merge(data_offset, how='left',left_index=True, right_index=True,\
                            suffixes=['','_prev_check'])

In [19]:
#Z stands for GMT and UTC timezone it is offset from 0 by coordinated time

df_gowalla['time_obj'] = pd.to_datetime(df_gowalla['checkin_time'].str.replace('[T]',':').str.replace('Z',''),\
                                        format='%Y-%m-%d:%H:%M:%S')

df_bright['time_obj'] = pd.to_datetime(df_bright['checkin_time'].str.replace('[T]',':').str.replace('Z',''),\
                                        format='%Y-%m-%d:%H:%M:%S')

In [20]:
df_gowalla.head(5)

,Unnamed: 0,user,user_house_lat,user_house_long,States of homes,geometry homes,checkin_time,latitude,longitude,country,name,date,tmin,tmax,pr,us_holiday,season,distance_from_home,time_obj
0,0,0,30.263544,-97.744633,Texas,POINT (-97.74463265845054 30.26354372623895),2010-10-19T23:55:27Z,30.235909,-97.795140,USA,Texas,2010-10-19,288.600006,302.100006,0.0,None,autumn_2010,5.742689,2010-10-19 23:55:27
1,1,0,30.263544,-97.744633,Texas,POINT (-97.74463265845054 30.26354372623895),2010-10-18T22:17:43Z,30.269103,-97.749395,USA,Texas,2010-10-18,289.100006,301.799988,0.0,None,autumn_2010,0.768984,2010-10-18 22:17:43
2,2,0,30.263544,-97.744633,Texas,POINT (-97.74463265845054 30.26354372623895),2010-10-17T23:42:03Z,30.255731,-97.763386,USA,Texas,2010-10-17,287.299988,301.799988,0.0,None,autumn_2010,1.999700,2010-10-17 23:42:03
3,3,0,30.263544,-97.744633,Texas,POINT (-97.74463265845054 30.26354372623895),2010-10-17T19:26:05Z,30.263418,-97.757597,USA,Texas,2010-10-17,287.299988,301.799988,0.0,None,autumn_2010,1.245154,2010-10-17 19:26:05
4,4,0,30.263544,-97.744633,Texas,POINT (-97.74463265845054 30.26354372623895),2010-10-16T18:50:42Z,30.274292,-97.740523,USA,Texas,2010-10-16,282.299988,301.299988,0.0,None,autumn_2010,1.258630,2010-10-16 18:50:42


In [21]:
df_gowalla_off = df_gowalla.sort_values('time_obj', ascending=True).groupby('user')\
                                                    .progress_apply(lambda group: offset(group))

100%|██████████| 49605/49605 [03:07<00:00, 264.37it/s]


In [22]:
df_bright_off = df_bright.sort_values('time_obj', ascending=True).groupby('user')\
                                                    .progress_apply(lambda group: offset(group))

100%|██████████| 29881/29881 [01:52<00:00, 264.57it/s]


In [23]:
df_gowalla_off = df_gowalla_off.reset_index(drop=True)
df_bright_off = df_bright_off.reset_index(drop=True)

In [24]:
df_gowalla_off['distance_from_prev_check'] = df_gowalla_off.parallel_apply(lambda x: dist_km(x['latitude'],\
                                                                     x['longitude'],\
                                                                     x['latitude_prev_check'],\
                                                                     x['longitude_prev_check']), axis=1)

<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arcco

In [25]:
df_bright_off['distance_from_prev_check'] = df_bright_off.parallel_apply(lambda x: dist_km(x['latitude'],\
                                                                     x['longitude'],\
                                                                     x['latitude_prev_check'],\
                                                                     x['longitude_prev_check']), axis=1)

<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arccos
  distance_km = np.arccos(np.sin(np.pi * Lat_start / 180.0) * np.sin(np.pi * Lat_end / 180.0) \
<ipython-input-14-1aab94a3e3b8>:8: RuntimeWarning: invalid value encountered in arcco

## Add Timezones & Local Time

In [27]:
df_bright_off['timezone'] = df_bright_off[['longitude', 'latitude']].progress_apply(lambda x:tf.timezone_at(lng=x[0],lat=x[1]),axis=1)
df_gowalla_off['timezone'] = df_gowalla_off[['longitude', 'latitude']].progress_apply(lambda x:tf.timezone_at(lng=x[0],lat=x[1]),axis=1)

100%|██████████| 3518110/3518110 [00:36<00:00, 95687.96it/s] 


In [28]:
df_gowalla_off['utc'] = df_gowalla_off['time_obj'].parallel_apply(timezone(utc.zone).localize)
df_bright_off['utc'] = df_bright_off['time_obj'].parallel_apply(timezone(utc.zone).localize)

In [29]:
def set_time_zone(row):
    try:
        return row[0].astimezone(timezone(row[1]))
    except:
        return None

In [30]:
df_gowalla_off['local_time'] = df_gowalla_off[['utc','timezone']].parallel_apply(set_time_zone,axis=1)
df_bright_off['local_time'] = df_bright_off[['utc','timezone']].parallel_apply(set_time_zone,axis=1)

In [31]:
df_gowalla_off['local_time_grouper'] = df_gowalla_off['local_time'].parallel_apply(lambda x: x.replace(tzinfo=None) if(pd.notnull(x)) else x )
df_bright_off['local_time_grouper'] = df_bright_off['local_time'].parallel_apply(lambda x: x.replace(tzinfo=None) if(pd.notnull(x)) else x)

In [32]:
df_gowalla_off.to_csv('./Data/all_feat_gowalla.csv', index=False)
df_bright_off.to_csv('./Data/all_feat_bright.csv', index=False)

In [ ]:
df_gowalla_off["distance_from_prev_check"]

In [ ]:
print(df_gowalla_off['distance_from_prev_check']

## Entropy

**Daily Movement Entropy (Average entropy of check in locations over time)**
- average shannon entropy of check in locations for each hour of the week
- lower the entropy, lower the variability of check ins during that time period
- need to recreate the graphs to make sure using correct shannon entropy

H(X) = -$\sum_{i=1}^{n} P(x_i)LogP(x_i)$

Took inspiration from this stack overflow for entropy
https://stackoverflow.com/questions/15450192/fastest-way-to-compute-entropy-in-python

#### TODO: Only take users who meet a certain criteria for number of check ins to avoid a bunch of 0 entropy

If we include location ID use this instead of unique loc

### Entropy 1

In [ ]:
def apply_entropy(data):
    p_data = data.value_counts()
    ent = stats.entropy(p_data, base=2)
    return ent

In [ ]:
df_bright2 = df_bright.copy()

In [ ]:
df_bright2['weekday'] = df_bright2['local_time_grouper'].map(lambda x: x.weekday())
df_bright2['hour'] = df_bright2['local_time_grouper'].map(lambda x: x.hour)

In [ ]:
df_bright2_group = df_bright2.groupby(['user','weekday','hour'])
#df_bright_group = df_bright.groupby(pd.Grouper(key="local_time_grouper", freq="1H"))

In [ ]:
entropy_bright2 = df_bright2_group['unique_loc'].parallel_apply(apply_entropy)
entropy_bright2 = entropy_bright2.to_frame().reset_index(drop=False)

In [ ]:
#entropy_bright2 = entropy_bright2[entropy_bright2['unique_loc'] > 0]

In [ ]:
entropy_mean_bright2 = entropy_bright2.drop('user', axis=1).groupby(['weekday','hour']).mean().reset_index(drop=False)

In [ ]:
entropy_mean_bright2['graph_date'] = entropy_mean_bright2.apply(lambda x: \
                                                          pd.Timestamp(year=2020,month=6,day=int(x['weekday']+1),\
                                                                       hour=int(x['hour'])),axis=1)

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
ax1.set(xlabel='', ylabel='Avg Entropy')
ax1.plot(entropy_mean_bright2.graph_date, entropy_mean_bright2['unique_loc'], color='g', marker = '.')
#ax1.plot(entropy_week_hour.graph_date, entropy_week_hour['unique_loc'], color='g', marker = '.')
#ax1.plot(entropy_hour.index, df_bright.Registered, color='b')

ax1.xaxis.set(
    major_locator=mdates.DayLocator(),
    major_formatter=mdates.DateFormatter("\n\n%A"),
    minor_locator=mdates.HourLocator((6 ,12, 18)),
    minor_formatter=mdates.DateFormatter("%H"),
)
plt.show()

### Entropy 2

In [ ]:
df_gowalla_group = df_gowalla.groupby(pd.Grouper(key="local_time_grouper", freq="1H"))
df_bright_group = df_bright.groupby(pd.Grouper(key="local_time_grouper", freq="1H"))

In [ ]:
entropy_hour_gowalla = df_gowalla_group['unique_loc'].parallel_apply(apply_entropy)
entropy_hour_gowalla = entropy_hour_gowalla.to_frame().reset_index(drop=False)

entropy_hour_bright = df_bright_group['unique_loc'].parallel_apply(apply_entropy)
entropy_hour_bright = entropy_hour_bright.to_frame().reset_index(drop=False)

In [ ]:
entropy_hour_gowalla['weekday'] = entropy_hour_gowalla['local_time_grouper'].map(lambda x: x.weekday())
entropy_hour_gowalla['hour'] = entropy_hour_gowalla['local_time_grouper'].map(lambda x: x.hour)

In [ ]:
entropy_hour_bright['weekday'] = entropy_hour_bright['local_time_grouper'].map(lambda x: x.weekday())
entropy_hour_bright['hour'] = entropy_hour_bright['local_time_grouper'].map(lambda x: x.hour)

In [ ]:
entropy_week_hour_gowalla = entropy_hour_gowalla.groupby(['weekday','hour']).mean()
entropy_week_hour_gowalla = entropy_week_hour_gowalla.reset_index(drop=False)

In [ ]:
entropy_week_hour_bright = entropy_hour_bright.groupby(['weekday','hour']).mean()
entropy_week_hour_bright = entropy_week_hour_bright.reset_index(drop=False)

In [ ]:
entropy_week_hour_gowalla['graph_date'] = entropy_week_hour_gowalla.apply(lambda x: \
                                                          pd.Timestamp(year=2020,month=6,day=int(x['weekday']+1),\
                                                                       hour=int(x['hour'])),axis=1)

In [ ]:
entropy_week_hour_bright['graph_date'] = entropy_week_hour_bright.apply(lambda x: \
                                                          pd.Timestamp(year=2020,month=6,day=int(x['weekday']+1),\
                                                                       hour=int(x['hour'])),axis=1)

In [ ]:
import matplotlib.dates as mdates

fig, ax1 = plt.subplots(figsize=(10, 5))
ax1.set(xlabel='', ylabel='Avg Entropy')
ax1.plot(entropy_week_hour_gowalla.graph_date, entropy_week_hour_gowalla['unique_loc'], color='g', marker = '.')
#ax1.plot(entropy_week_hour.graph_date, entropy_week_hour['unique_loc'], color='g', marker = '.')
#ax1.plot(entropy_hour.index, df_bright.Registered, color='b')

ax1.xaxis.set(
    major_locator=mdates.DayLocator(),
    major_formatter=mdates.DateFormatter("\n\n%A"),
    minor_locator=mdates.HourLocator((6 ,12, 18)),
    minor_formatter=mdates.DateFormatter("%H"),
)
plt.show()